In [1]:
#Grab Data
import yfinance as yf

#Usual Suspects
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# plt.style.use("seaborn-v0_8-deep")

import plotly.express as px
from pypfopt.plotting import plot_weights
import seaborn as sns

# Use PyPortfolioOpt for Calculations
from pypfopt import EfficientFrontier, objective_functions
from pypfopt import black_litterman, risk_models
from pypfopt import BlackLittermanModel, plotting
from pypfopt import DiscreteAllocation

In [2]:
#Create a Portfolio
symbols = [
    'AAPL',
    'MSFT',
    'META',
    'AMZN',
    'XOM',
    'UNH',
    'JNJ',
    'V',
    'HD',
    'ABBV',
    'KO',
    'DIS',
    'T',
    'UPS',
    'LMT',
    'CAT',
    'F',
    'MAR',
    'O',
    'HSY'
]

In [3]:
#Get the stock data
portfolio = yf.download(symbols, start="2000-01-01", end="2023-12-31")['Adj Close']
portfolio.head()

[*********************100%***********************]  20 of 20 completed


Ticker,AAPL,ABBV,AMZN,CAT,DIS,F,HD,HSY,JNJ,KO,LMT,MAR,META,MSFT,O,T,UNH,UPS,V,XOM
Date,,,,,,,,,,,,,,,,,,,,
2000-01-03 00:00:00+00:00,0.844004,NaN,4.468750,12.809785,22.934355,12.503567,39.068657,13.221469,24.316019,14.122949,10.843440,11.312893,NaN,35.935867,2.545421,6.692317,5.358777,34.991390,NaN,18.035826
2000-01-04 00:00:00+00:00,0.772846,NaN,4.096875,12.645138,24.277790,12.085781,37.008453,12.722550,23.425812,14.138607,11.211580,11.243352,NaN,34.721939,2.514753,6.300744,5.290233,33.882622,NaN,17.690359
2000-01-05 00:00:00+00:00,0.784155,NaN,3.487500,12.941502,25.285368,12.130543,37.757633,12.776011,23.673067,14.263863,11.278512,11.428805,NaN,35.088039,2.499418,6.398637,5.277772,35.480545,NaN,18.654768
2000-01-06 00:00:00+00:00,0.716296,NaN,3.278125,13.600108,24.277790,12.145466,35.959648,13.114557,24.414930,14.279512,11.445853,11.591083,NaN,33.912682,2.530087,6.274661,5.470937,35.480545,NaN,19.619169
2000-01-07 00:00:00+00:00,0.750226,NaN,3.478125,14.044664,23.893951,13.040709,38.057281,13.150193,25.453508,15.218962,11.479316,11.915634,NaN,34.355850,2.576090,6.328442,6.112743,34.632660,NaN,19.561590


In [4]:
#SP500 ETF Benchmark
market_prices = yf.download("SPY", start='2000-01-01', end='2023-12-31')["Adj Close"]
market_prices.head()

[*********************100%***********************]  1 of 1 completed


Date
2000-01-03    93.004875
2000-01-04    89.367851
2000-01-05    89.527748
2000-01-06    88.088852
2000-01-07    93.204758
Name: Adj Close, dtype: float64

In [5]:
mcaps = {}

for t in symbols:
    stock = yf.Ticker(t)
    mcaps[t] = stock.info["marketCap"]

mcaps

# Getting Priors

In [ ]:
#Calculate Sigma and Delta to get implied market returns
#Ledoit-Wolf is a particular form of shrinkage, where the shrinkage coefficient is computed using O?
S = risk_models.CovarianceShrinkage(portfolio).ledoit_wolf()

delta = black_litterman.market_implied_risk_aversion(market_prices)
delta

In [ ]:
#Visualize the Covariant Correlation
sns.heatmap(S.corr(), cmap='coolwarm')

In [ ]:
market_prior = black_litterman.market_implied_prior_returns(mcaps, delta, S)
market_prior

In [ ]:
#What am I looking at here?
market_prior.plot.barh(figsize=(10,5));

# Integrating Views

In [36]:
#You don't have to provide views on all the assets
viewdict = {
    'AAPL':0.10,
    'MSFT':0.10,
    'META':0.05,
    'AMZN':0.30,
    'XOM':0.02,
    'UNH':0.01,
    'JNJ':0.15,
    'V':0.09,
    'HD':0.16,
    'ABBV':0.07,
    'KO':0.01,
    'DIS':-0.23,
    'T':0.16,
    'UPS':0.10,
    'LMT':-0.09,
    'CAT':0.30,
    'F':0.16,
    'MAR':-0.08,
    'O':0.30,
    'HSY':-0.26
}

bl = BlackLittermanModel(S, pi=market_prior, absolute_views=viewdict)

## Creating Confidences

In [37]:
intervals = [
    (0, 0.25),
    (0.1, 0.4),
    (-0.1, 0.15),
    (-0.05, 0.1),
    (0.15, 0.25),
    (-0.1, 0),
    (0.1, 0.2),
    (0.08, 0.12),
    (0.1, 0.9),
    (0, 0.3),
    (0, 0.25),
    (0.1, 0.4),
    (-0.1, 0.15),
    (-0.05, 0.1),
    (0.15, 0.25),
    (-0.1, 0),
    (0.1, 0.2),
    (0.08, 0.12),
    (0.1, 0.9),
    (0, 0.3),
]

In [ ]:
variances = []
for lb, ub in intervals:
    sigma = (ub - lb)/2
    variances.append(sigma ** 2)

print(variances)
omega = np.diag(variances)

# Calculate Posterior Estimate Returns

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
im = ax.imshow(omega)

# We want to show all ticks...
ax.set_xticks(np.arange(len(bl.tickers)))
ax.set_yticks(np.arange(len(bl.tickers)))

ax.set_xticklabels(bl.tickers)
ax.set_yticklabels(bl.tickers)
plt.show()

In [40]:
# We are using the shortcut to automatically compute market-implied prior
bl = BlackLittermanModel(S, pi="market", market_caps=mcaps, risk_aversion=delta,
                        absolute_views=viewdict, omega=omega)

In [ ]:
# Posterior estimate of returns
ret_bl = bl.bl_returns()
ret_bl 

In [ ]:
rets_df = pd.DataFrame([market_prior, ret_bl, pd.Series(viewdict)],
             index=["Prior", "Posterior", "Views"]).T
rets_df

In [ ]:
rets_df.plot.bar(figsize=(12,8));

In [ ]:
S_bl = bl.bl_cov()
plotting.plot_covariance(S_bl);

# Portfolio Allocation

In [ ]:
ef = EfficientFrontier(ret_bl, S_bl)
ef.add_objective(objective_functions.L2_reg)
ef.max_sharpe()
weights = ef.clean_weights()
weights

In [ ]:
pd.Series(weights).plot.pie(figsize=(9,9));

In [ ]:
# Maximum Sharpe
ef = EfficientFrontier(ret_bl, S_bl)
ef.add_objective(objective_functions.L2_reg)
ef.max_sharpe()
weights = ef.clean_weights()

plot_weights(weights)
ef.portfolio_performance(verbose = True, risk_free_rate = 0.009)